# Fuzzy Cleaning Systems: The Cleaner Solution to your Comfort

Cleaning systems are commonly used to steer robotic support enteties like vacuum cleaners with or without wet cleaning. The used, clean/dirty segmentation approach faces several problems due to it's inability to notice different factors that humans encounter when thinking of a "clean" flat or home.

Based on Fuzzy Control Systems, we can build an adapted system for Fuzzy cleaning.

In [ ]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# New Antecedent/Consequent objects for the cleaning problem
dirtiness = ctrl.Antecedent(np.arange(0, 11, 1), 'dirtiness')  # Level of dirtiness from 0 (clean) to 10 (very dirty)
room_size = ctrl.Antecedent(np.arange(0, 101, 1), 'room_size')  # Size of the room in square meters
cleaning_intensity = ctrl.Consequent(np.arange(0, 101, 1), 'cleaning_intensity')  # Cleaning intensity from 0 to 100%

# Auto-membership function population for dirtiness and room_size (low, medium, high)
dirtiness.automf(3)
room_size.automf(3)

# Custom membership functions for cleaning intensity (low, medium, high)
cleaning_intensity['low'] = fuzz.trimf(cleaning_intensity.universe, [0, 0, 50])
cleaning_intensity['medium'] = fuzz.trimf(cleaning_intensity.universe, [25, 50, 75])
cleaning_intensity['high'] = fuzz.trimf(cleaning_intensity.universe, [50, 100, 100])

# Define the rules for controlling cleaning intensity based on dirtiness and room size
rule1 = ctrl.Rule(dirtiness['poor'] | room_size['poor'], cleaning_intensity['low'])
rule2 = ctrl.Rule(dirtiness['average'] & room_size['average'], cleaning_intensity['medium'])
rule3 = ctrl.Rule(dirtiness['good'] | room_size['good'], cleaning_intensity['high'])

# Create a control system and simulation
cleaning_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
cleaning_simulation = ctrl.ControlSystemSimulation(cleaning_ctrl)

# Set input values for dirtiness and room_size
cleaning_simulation.input['dirtiness'] = 7  # Example: moderately dirty
cleaning_simulation.input['room_size'] = 30  # Example: a small room

# Compute the cleaning intensity based on inputs
cleaning_simulation.compute()

# Output the result
print(f"Recommended cleaning intensity: {cleaning_simulation.output['cleaning_intensity']}")

# Visualize the membership functions (optional)
dirtiness.view()
room_size.view()
cleaning_intensity.view()


# Fuzzy Rules for Cleaning Intensity

To make these fuzzy membership functions useful, we define the fuzzy relationships between the input (dirtiness and room size) and the output (cleaning intensity). For the purposes of our example, consider three simple rules:

1. **If the dirtiness is poor OR the room size is small, then the cleaning intensity will be low.**
2. **If the dirtiness is average AND the room size is medium, then the cleaning intensity will be medium.**
3. **If the dirtiness is high OR the room size is large, then the cleaning intensity will be high.**

These rules mirror how we would expect a robot vacuum to adjust its cleaning effort based on the condition of the room. However, the rules are inherently fuzzy—mapping these imprecise conditions into a well-defined cleaning intensity is where fuzzy logic excels. It helps bridge the gap between vague inputs and actionable cleaning strategies.

In [ ]:
# Define the fuzzy rules for cleaning intensity
rule1 = ctrl.Rule(dirtiness['poor'] | room_size['poor'], cleaning_intensity['low'])  # If dirtiness is low OR room is small, cleaning intensity is low
rule2 = ctrl.Rule(dirtiness['average'] & room_size['average'], cleaning_intensity['medium'])  # If dirtiness is average AND room size is medium, cleaning intensity is medium
rule3 = ctrl.Rule(dirtiness['good'] | room_size['good'], cleaning_intensity['high'])  # If dirtiness is high OR room is large, cleaning intensity is high

# Visualize one of the rules (optional)
rule1.view()

# Control System Creation and Simulation

Now that we have our rules defined, we can simply create a control system via:

In [ ]:
cleaning_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])

In order to simulate this control system, we will create a ControlSystemSimulation. Think of this object as representing our controller applied to a specific set of cleaning conditions. For cleaning, this might be for a small, lightly soiled living room. We would create another ControlSystemSimulation when we’re trying to apply our cleaning_ctrl for a large, heavily soiled kitchen, because the inputs (dirtiness and room size) would be different.

In [ ]:
cleaning = ctrl.ControlSystemSimulation(cleaning_ctrl)

We can now simulate our control system by simply specifying the inputs and calling the compute method. Suppose the room has a dirtiness level of 7 out of 10 and the room size is 30 square meters.

In [ ]:
# Pass inputs to the ControlSystem using Antecedent labels with Pythonic API
# Note: if you prefer passing many inputs all at once, use .inputs(dict_of_data)
cleaning_simulation.input['dirtiness'] = 7  # Example: moderately dirty room
cleaning_simulation.input['room_size'] = 30  # Example: room size in square meters

# Crunch the numbers
cleaning_simulation.compute()

# Once computed, we can view the result as well as visualize it.
print(f"Recommended cleaning intensity: {cleaning_simulation.output['cleaning_intensity']}")
cleaning_intensity.view(sim=cleaning_simulation)

# Defuzzification

Fuzzy logic calculations are excellent tools, but to use them the fuzzy result must be converted back into a single number. This is known as defuzzification.
There are several possible methods for defuzzification, exposed via skfuzzy.defuzz.

In [ ]:
import matplotlib.pyplot as plt

# Generate trapezoidal membership function on range [0, 10] for dirtiness
x = np.arange(0, 10.05, 0.1)
mfx = fuzz.trapmf(x, [2, 3, 7, 9])  # Example trapezoidal membership function for dirtiness levels

# Defuzzify this membership function five ways
defuzz_centroid = fuzz.defuzz(x, mfx, 'centroid')  # Same as skfuzzy.centroid
defuzz_bisector = fuzz.defuzz(x, mfx, 'bisector')
defuzz_mom = fuzz.defuzz(x, mfx, 'mom')
defuzz_som = fuzz.defuzz(x, mfx, 'som')
defuzz_lom = fuzz.defuzz(x, mfx, 'lom')

# Collect info for vertical lines
labels = ['centroid', 'bisector', 'mean of maximum', 'min of maximum', 'max of maximum']
xvals = [defuzz_centroid, defuzz_bisector, defuzz_mom, defuzz_som, defuzz_lom]
colors = ['r', 'b', 'g', 'c', 'm']
ymax = [fuzz.interp_membership(x, mfx, i) for i in xvals]

# Display and compare defuzzification results against membership function
plt.figure(figsize=(8, 5))

plt.plot(x, mfx, 'k')
for xv, y, label, color in zip(xvals, ymax, labels, colors):
    plt.vlines(xv, 0, y, label=label, color=color)
plt.ylabel('Fuzzy membership')
plt.xlabel('Dirtiness level (0 to 10)')
plt.ylim(-0.1, 1.1)
plt.legend(loc=2)

plt.show()